<a href="https://colab.research.google.com/github/akshatvishu/DL-notes/blob/main/VAE_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assume we have a training data $(x^{i})_{i=1}^{N}$ is generated from an latent representation z.

Intution: 

x = image, 
z is latent factor used to generate x:attributes,orientation etc.


1. Learn latent features z from the raw data
2. Sample fromt the model to generate new data
3. Assume simple prior p(z), eg: gaussian
4. Represent p(x|z) with a NN.

After training, sample new data :

$$p_Θ*(x|z^{(i)})$$

* sample z from prior $p_Θ*(z)$

* During training:-
z -----> x



Decoder must be probabilistic:

* Decoder inputs z, outputs mean $μ_{x|z}$ and diagonal covariance $∑_{x|z}$ .

* Sampel x from gaussian with mean $μ_{x|z} $ and diagonal covariance $ ∑_{x|z} $



How to train model?
Basic Idea: Max. Likehood of data
If we could observe z for each x, then we could train a conditional generative model p(x|z) .


We don't observe z, so we need to marginalize it
Joint distribution over z and x and then we can further broken it down to condtional prob. of x given z times the prior distribution of z (using chain rule) and this prior we assume that is gaussian. But the integral is intractable to compute as z is some higher dimension vector space .

Formula : Density over x
$$p_Θ(x)=∫p_Θ(x,z)dz=∫p_Θ(x|z)p_Θ(z)dz $$


Thus, let's use Bayes' Rule:
$$ p_θ(x)=\frac{p_θ(x|z)p_θ(z)}{p_θ(z|x)} $$


But here still we can't calculate $p_θ(z|x)$ 

Solution: Train another networ (encoder) that learns 

$$ 
q_Φ(z|x)≈p_θ(z|x) 
$$
Thus,


$$ 
\mathbb p_θ(x)=\frac{p_θ(x|z)p_θ(z)}{p_θ(z|x)} ≈  p_θ(x)=\frac{p_θ(x|z)p_θ(z)}{q_Φ(z|x)}
 $$

Encoder Network inputs data x, gives distribution over latent codes z

$$
\mathbb q_Φ(z|x)= N(μ_{z|x},∑_{z|x}) 
$$

Decoder Network inputs latent code z, gives distribution over data x

$$
\mathbb p_θ(x|z)= N(μ_{x|z},∑_{x|z}) 
$$

Idea: Jointly train both encoder and decoder

$$
\mathbb \log p_θ(x) = \log \frac {p_θ(x|z)p(z)}{p_θ(z|x)} = \log \frac {p_θ(x|z)p(z)q_Φ(z|x)}{p_θ(z|x)q_Φ(z|x)}
$$

Multiply top and bottom by $q_Φ(z|x)$

Using logarithms to split up the terms:

$$
\mathbb =\log p_θ(x|z) - \log\frac{q_Φ(z|x)}{p(z)}+\log\frac{q_Φ(z|x)}{p_θ(z|x)}
$$

Now, we can see that $\log p_θ(x)$ does not depend upon z.Also, whenever we have a random variable that isn't dependent on another random variable then we can take the expectation over that whole thing.

$$
\mathbb \log p_θ(x)=\mathbb{E}_{z \sim q_Φ(z|x)}\left[\log p_θ(x)\right]
$$

$$
\mathbb \log p_θ(x)=\mathbb {E_z}\left[\log p_θ(x|z)\right]-\mathbb{E}_{z}\left[\log \frac{q_Φ(z|x)}{p(z)}\right]+\mathbb{E}_{z}\left[\log \frac{q_Φ(z|x)}{p_θ(z|x)}\right]
$$


$$
=\color{yellow}{\mathbb{E}_{z \sim q_Φ(z|x)}\left[\log p_θ(x|z)\right]}-\color{green}{\mathbb{D_{KL}\left(q_Φ(z|x),p(z)\right)}+\color{red}{\mathbb{D_{KL}\left(q_Φ(z|x),p_θ(z|x)\right)}}}
$$

* Yellow - data reconstruction
* Green  - KL divergence b/w prior and samples from the encoder network
* Red    - KL divergence b/w encoder and posterior of decoder 

Now, the red term is the one we cannot compute becuase it involves $p_θ(z|x)$ but we can compute the yellow and the green terms. Also, we know that KL >=0, so dropping this last red term gives a **lower bound** on the data likelihood:

$$
\mathbb \log p_θ(x) \geq \color{yellow}{\mathbb{E}_{z \sim q_Φ(z|x)}\left[\log p_θ(x|z)\right]}-\color{green}{\mathbb{D_{KL}\left(q_Φ(z|x),p(z)\right)}}
$$

On the left, we have the actual true density of the data under the probalistic model that we setup and on the right we have the lower bound to that density that contains a reconstruction term and a KL divergence term.

Thus, we jointly want to train encoder q and decoder p to maximize the **variational lower bound** on the data likehood(Trick:Variationl Inference).




